In [23]:
# Importer les données: json + correspondance uniprot/refseq + genomic
import json
import pandas as pd

def merge_two_dicts(x, y):
    z = x.copy()
    z.update(y)    
    return z

def importJson(filePath):
    data_dict = []
    with open(filePath) as json_data:
        for i in json_data:
            data_dict.append(json.loads(i))
    MyJsonFull = data_dict[0]
    for i in data_dict[1:]:
        MyJsonFull = merge_two_dicts(MyJsonFull, i)
    return MyJsonFull

def catExonPos(MyJsonFull, seqID, dict_Genomic):
    exonString = []
    if MyJsonFull[seqID]["strand"] == -1:
        ABSOLUTE_POST = MyJsonFull[seqID]["end"]
        exon_list = MyJsonFull[seqID]["Exon"]
        counter = 1
        for i in exon_list:
            stop = str(abs(i['end']-ABSOLUTE_POST))
            start = str(abs(i['start']-ABSOLUTE_POST)+1)
            res = [val for key, val in my_Genomic.items() if seqID in key]
            exonString.append(seqID+ "\tExon\t"+str(counter)+"\t"+stop+"\t"+start+"\t"+res[0][int(stop):int(start)])
            counter+=1

    if MyJsonFull[seqID]["strand"] == 1:
        ABSOLUTE_POST = MyJsonFull[seqID]["start"]
        exon_list = MyJsonFull[seqID]["Exon"]
        counter = 1
        for i in exon_list:
            start = str(abs(i['start']-ABSOLUTE_POST))
            stop = str(abs(i['end']-ABSOLUTE_POST)+1)
            res = [val for key, val in dict_Genomic.items() if seqID in key]
            exonString.append(seqID+ "\tExon\t"+str(counter)+"\t"+start+"\t"+stop+"\t"+res[0][int(start):int(stop)])
            counter+=1
    return(exonString)
    
def fasta2List(pathFasta):
    f = open(pathFasta, "r")
    title = []
    seq = []
    seq_temp = []
    for line in f:
        if line[0] == ">":
            seq.append(''.join(seq_temp).replace("\n", ""))
            title.append(line.replace("\n", ""))
            seq_temp = []
        else:
            seq_temp.append(line)
    seq.append(''.join(seq_temp).replace("\n", ""))
    seq.pop(0)
    dictionary = dict(zip(title, seq))
    return dictionary

In [3]:
ID_file = pd.read_csv("../uniprot-to-exon/transcript_ensembl.tab", sep = "\t")
jsonFile = importJson('C:/Users/Glados/Documents/stage-thompson/uniprot-to-exon/lookup_newline.json')
my_Genomic = fasta2List("../uniprot-to-exon/genomics_new.fa")

In [24]:
with open("../uniprot-to-exon/Exon_map.tab", "w") as exon_file:
    for i in ID_file.iloc[:,1]:
        uniprot_name = ID_file.loc[ID_file["To"] == i].iloc[0,0]
        for exon in catExonPos(jsonFile, i, my_Genomic):
            exon_file.write(uniprot_name+"\t"+exon+"\n")

In [22]:
jsonFile["ENSPPYT00000024082"]

for i in catExonPos(jsonFile, "ENSPPYT00000024082", my_Genomic):
    print(i)

jsonFile["ENSPPYT00000024082"]["Translation"]

ENSPPYT00000024082	Exon	1	19	443	ATGCGTCCTAAGAACCCCTTACACTCGCGGCACAGGCTGACAGTATTGTCGGGGGCTATTCTCTCTCCCAGGAACATGGCGGCGAGTCAGGGAGGAGGTGGTAACAGTGGGGGCGGCGGTTGTGGTGGAGGTGGAAGTAGCGGTGGCTGTGGCACGGCTGGAGGGGGAGGTGGCGGGACTGGAGGGGGAGGCGGCGGCGGCGGCGGGACCCTGGTGGTCCCCATCCCGGTACCGACTCTTTTCGGTCAGCCGTTCCCCAACGGGCCGCCGTGGAACCCGGGGAGCCTGCAGCCTCAGCACACCGTGAGGAGCCTGGACCGGGCCCTGGAAGAGGCGGGCAGCTCCGGTATTCTGAGCCTCAGTGGTCGGAAACTCCGAGACTTCCCGGGCAGCGGCTACGACCTGACGGACACCACCCAAGCAG
ENSPPYT00000024082	Exon	2	44602	44747	ATCTTTCCAGAAATCGTTTTACAGAAATCCCTTCTGATGTCTGGTTATTTGCACCCCTTGAAACATTAAATTTATATCATAATTGCATCAAAACCATTCCTGAAGCCATTAAAAATCTGCAGATGTTAACATACCTTAACATTAG
ENSPPYT00000024082	Exon	3	48437	48564	CCGAAATCTTTTATCAACATTGCCAAAATACCTATTTGATCTTCCCCTTAAAGTTTTGGTCATCAGTAATAATAAACTGGTATCCATTCCAGAAGAAATTGGGAAGTTAAAAGATTTAATGGAATTG
ENSPPYT00000024082	Exon	4	53258	53364	GATATTAGCTGCAATGAGATTCAAGTCCTTCCCCAACAAATGGGAAAATTACATTCACTTAGAGAGCTAAATATAAGAAGAAATAATCTTCGTGTTTTGCCAGATG
ENSPPYT00000024082	Exon	5	53446	53583	AATTAGGAG

{'id': 'ENSPPYP00000023112',
 'Parent': 'ENSPPYT00000024082',
 'length': 767,
 'db_type': 'core',
 'start': 114060524,
 'species': 'pongo_abelii',
 'end': 114180146,
 'object_type': 'Translation'}